<a href="https://colab.research.google.com/github/Norceis/radar-project/blob/main/radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, concatenate, Activation, Concatenate, Conv2DTranspose, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
with open('drive/MyDrive/data_slices2.pickle', 'rb') as f:
    x, y = pickle.load(f)

x = x[..., np.newaxis] #slices

print(x.shape, y.shape)

img_shape = x[0].shape

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

(2928, 54, 216, 1) (2928,)


In [7]:
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=img_shape))
model.add(MaxPooling2D())
model.add(LeakyReLU())
model.add(Dropout(0.3))

model.add(Conv2D(32, (3,3), strides=(2, 2), padding='same'))
model.add(MaxPooling2D())
model.add(LeakyReLU())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
model.add(MaxPooling2D())
model.add(LeakyReLU())
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512))
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

slice = Input(shape=img_shape)
pred = model(slice)
radar_model = Model(slice, pred)


METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


# optimizer = Adam(0.0002, 0.5)
radar_model.compile(loss='binary_crossentropy',
                          optimizer='Adam',
                        metrics=METRICS)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 54, 216, 16)       160       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 27, 108, 16)      0         
 2D)                                                             
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 27, 108, 16)       0         
                                                                 
 dropout_8 (Dropout)         (None, 27, 108, 16)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 54, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 27, 32)        0         
 2D)                                                  

In [8]:
radar_model.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
74/74 [==============================] - 3s 9ms/step - loss: 0.4175 - tp: 1765.0000 - fp: 347.0000 - tn: 173.0000 - fn: 57.0000 - accuracy: 0.8275 - precision: 0.8357 - recall: 0.9687 - auc: 0.8009 - prc: 0.9201
Epoch 2/20
74/74 [==============================] - 1s 9ms/step - loss: 0.2422 - tp: 1788.0000 - fp: 151.0000 - tn: 369.0000 - fn: 34.0000 - accuracy: 0.9210 - precision: 0.9221 - recall: 0.9813 - auc: 0.9245 - prc: 0.9732
Epoch 3/20
74/74 [==============================] - 1s 11ms/step - loss: 0.1901 - tp: 1807.0000 - fp: 116.0000 - tn: 404.0000 - fn: 15.0000 - accuracy: 0.9441 - precision: 0.9397 - recall: 0.9918 - auc: 0.9428 - prc: 0.9792
Epoch 4/20
74/74 [==============================] - 1s 11ms/step - loss: 0.1711 - tp: 1809.0000 - fp: 107.0000 - tn: 413.0000 - fn: 13.0000 - accuracy: 0.9488 - precision: 0.9442 - recall: 0.9929 - auc: 0.9522 - prc: 0.9833
Epoch 5/20
74/74 [==============================] - 1s 12ms/step - loss: 0.1688 - tp: 1807.0000 - fp: 109.

In [9]:
radar_model.evaluate(x_test, y_test)


19/19 [==============================] - 1s 12ms/step - loss: 0.1923 - tp: 464.0000 - fp: 17.0000 - tn: 89.0000 - fn: 16.0000 - accuracy: 0.9437 - precision: 0.9647 - recall: 0.9667 - auc: 0.9519 - prc: 0.9835


[0.1923338919878006,
 464.0,
 17.0,
 89.0,
 16.0,
 0.9436860084533691,
 0.9646569490432739,
 0.9666666388511658,
 0.9519261121749878,
 0.9834976196289062]

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=int32), array([ 626, 2302]))

In [ ]:
626/(626 + 2302), 2302/(626 + 2302)

(0.21379781420765026, 0.7862021857923497)